# Conflict & Adaptation Analysis
## How Conflict Handling Relates to Team Outcomes


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Load data
df = pd.read_excel('SurveyData211.xlsx')

# Key variables
OUTCOME_VARS = ['GO1', 'NPS1', 'NPS2', 'SE1', 'SE2', 'RLS1']
VAR_LABELS = {
    'CA1': 'Conflict Adaptation (1-10)',
    'GO1': 'Growth/Outcomes (1-5)',
    'NPS1': 'Team Satisfaction 1 (1-5)',
    'NPS2': 'Team Satisfaction 2 (1-5)',
    'SE1': 'Self-Efficacy 1 (1-5)',
    'SE2': 'Self-Efficacy 2 (1-5)',
    'RLS1': 'Relationship Strength (1-10)',
    'Section': 'Class Section'
}

print(f"Dataset: {df.shape[0]} responses, {df.shape[1]} variables")


## 1. Distribution of Conflict Adaptation Scores


In [ ]:
# Summary statistics for CA1 and outcome variables
summary_vars = ['CA1'] + OUTCOME_VARS
summary_stats = df[summary_vars].describe().round(2)
summary_stats


In [ ]:
# Figure 1: Distribution of Conflict Adaptation Scores
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram
axes[0].hist(df['CA1'].dropna(), bins=range(5, 12), edgecolor='white', color='#2E86AB', alpha=0.8)
axes[0].set_xlabel('Conflict Adaptation Score')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of CA1 Scores')
axes[0].set_xticks(range(6, 11))

# Box plot
axes[1].boxplot(df['CA1'].dropna(), vert=True)
axes[1].set_ylabel('Conflict Adaptation Score')
axes[1].set_title('CA1 Score Distribution')
axes[1].set_xticklabels(['CA1'])

plt.tight_layout()
plt.show()


## 2. Correlation Analysis: CA1 vs Outcome Variables


In [ ]:
# Correlation analysis
corr_vars = ['CA1'] + OUTCOME_VARS
corr_matrix = df[corr_vars].corr().round(3)

# Figure 2: Simple bar chart of CA1 correlations
ca_corrs = corr_matrix['CA1'].drop('CA1').sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(8, 5))
colors = ['#2E86AB' if r > 0 else '#E94F37' for r in ca_corrs.values]
bars = ax.barh(ca_corrs.index, ca_corrs.values, color=colors)
ax.set_xlabel('Correlation with Conflict Adaptation (CA1)')
ax.set_title('How CA1 Relates to Team Outcomes')
ax.axvline(x=0, color='black', linewidth=0.5)
ax.set_xlim(-0.1, 0.7)

# Add value labels
for bar, val in zip(bars, ca_corrs.values):
    ax.text(val + 0.02, bar.get_y() + bar.get_height()/2, f'{val:.2f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# Table: CA1 correlations with outcome variables
ca_corrs = corr_matrix['CA1'].drop('CA1').sort_values(ascending=False)
corr_table = pd.DataFrame({
    'Variable': ca_corrs.index,
    'Correlation with CA1': ca_corrs.values,
    'Interpretation': ['Strong positive' if abs(r) > 0.5 else 'Moderate positive' if r > 0.3 
                       else 'Weak positive' if r > 0 else 'Weak negative' for r in ca_corrs.values]
})
corr_table


## 3. CA1 vs Outcome Variables: Detailed Relationships


In [ ]:
# Figure 3: CA1 vs Growth/Outcomes (GO1)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Scatter with jitter
jitter = np.random.normal(0, 0.08, len(df))
axes[0].scatter(df['CA1'] + jitter, df['GO1'] + jitter, alpha=0.5, c='#2E86AB', s=50)
z = np.polyfit(df['CA1'].dropna(), df.loc[df['CA1'].notna(), 'GO1'], 1)
p = np.poly1d(z)
x_line = np.linspace(df['CA1'].min(), df['CA1'].max(), 100)
axes[0].plot(x_line, p(x_line), 'r--', linewidth=2, label=f'Trend (r={corr_matrix.loc["CA1","GO1"]:.2f})')
axes[0].set_xlabel('Conflict Adaptation (CA1)')
axes[0].set_ylabel('Growth/Outcomes (GO1)')
axes[0].set_title('CA1 vs GO1: Scatter Plot')
axes[0].legend()

# Box plot by CA1 groups
df['CA1_group'] = pd.cut(df['CA1'], bins=[5, 7, 8, 10], labels=['Low (6-7)', 'Mid (8)', 'High (9-10)'])
df.boxplot(column='GO1', by='CA1_group', ax=axes[1])
axes[1].set_xlabel('Conflict Adaptation Group')
axes[1].set_ylabel('Growth/Outcomes (GO1)')
axes[1].set_title('GO1 by CA1 Level')
plt.suptitle('')

plt.tight_layout()
plt.show()


In [ ]:
# Table: Team Satisfaction by CA1 Level
nps_by_group = df.groupby('CA1_group')['NPS1'].agg(['mean', 'count']).round(2)
nps_by_group.columns = ['Avg Satisfaction', 'N']
print("Team Satisfaction (NPS1) by Conflict Adaptation Level:")
display(nps_by_group)

# Figure 4: Simple bar - Satisfaction by CA1 level
fig, ax = plt.subplots(figsize=(8, 5))
groups = ['Low (6-7)', 'Mid (8)', 'High (9-10)']
nps_means = [nps_by_group.loc[g, 'Avg Satisfaction'] if g in nps_by_group.index else 0 for g in groups]

bars = ax.bar(groups, nps_means, color=['#E94F37', '#F4A261', '#2E86AB'])
ax.set_xlabel('Conflict Adaptation Level')
ax.set_ylabel('Average Team Satisfaction')
ax.set_title('Team Satisfaction Increases with Better Conflict Handling')
ax.set_ylim(0, 5)

for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
            f'{bar.get_height():.2f}', ha='center', fontsize=11)

plt.tight_layout()
plt.show()


In [ ]:
# Table: Self-Efficacy by CA1 Level
se_by_group = df.groupby('CA1_group')[['SE1', 'SE2']].mean().round(2)
se_by_group['SE Average'] = ((se_by_group['SE1'] + se_by_group['SE2']) / 2).round(2)
print("Self-Efficacy Scores by Conflict Adaptation Level:")
display(se_by_group)

# Figure 5: Simple grouped bar - SE by CA1 level
fig, ax = plt.subplots(figsize=(8, 5))
groups = ['Low (6-7)', 'Mid (8)', 'High (9-10)']
se_means = [se_by_group.loc[g, 'SE Average'] if g in se_by_group.index else 0 for g in groups]

bars = ax.bar(groups, se_means, color=['#E94F37', '#F4A261', '#2E86AB'])
ax.set_xlabel('Conflict Adaptation Level')
ax.set_ylabel('Average Self-Efficacy Score')
ax.set_title('Self-Efficacy Increases with Better Conflict Handling')
ax.set_ylim(0, 5)

for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
            f'{bar.get_height():.2f}', ha='center', fontsize=11)

plt.tight_layout()
plt.show()


In [ ]:
# Table: Relationship Strength by CA1 Level
rls_by_group = df.groupby('CA1_group')['RLS1'].agg(['mean', 'count']).round(2)
rls_by_group.columns = ['Avg Relationship Strength', 'N']
print("Relationship Strength (RLS1) by Conflict Adaptation Level:")
display(rls_by_group)

# Figure 6: Simple bar - Relationship Strength by CA1 level
fig, ax = plt.subplots(figsize=(8, 5))
groups = ['Low (6-7)', 'Mid (8)', 'High (9-10)']
rls_means = [rls_by_group.loc[g, 'Avg Relationship Strength'] if g in rls_by_group.index else 0 for g in groups]

bars = ax.bar(groups, rls_means, color=['#E94F37', '#F4A261', '#2E86AB'])
ax.set_xlabel('Conflict Adaptation Level')
ax.set_ylabel('Average Relationship Strength (1-10)')
ax.set_title('Stronger Relationships in Teams with Better Conflict Handling')
ax.set_ylim(0, 10)

for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, 
            f'{bar.get_height():.1f}', ha='center', fontsize=11)

plt.tight_layout()
plt.show()


## 4. High vs Low Conflict Adaptation: Group Comparison


In [ ]:
# Split by median CA1 score
ca_median = df['CA1'].median()
df['CA1_binary'] = df['CA1'].apply(lambda x: 'High CA' if x >= ca_median else 'Low CA')

# Comparison table
comparison = df.groupby('CA1_binary')[OUTCOME_VARS].agg(['mean', 'std']).round(2)
comparison.columns = [f'{col[0]}_{col[1]}' for col in comparison.columns]

# Reshape for cleaner display
comparison_clean = pd.DataFrame({
    'Group': ['High CA (≥{:.0f})'.format(ca_median), 'Low CA (<{:.0f})'.format(ca_median)],
    'N': [len(df[df['CA1_binary'] == 'High CA']), len(df[df['CA1_binary'] == 'Low CA'])],
    'GO1 Mean': [comparison.loc['High CA', 'GO1_mean'], comparison.loc['Low CA', 'GO1_mean']],
    'NPS1 Mean': [comparison.loc['High CA', 'NPS1_mean'], comparison.loc['Low CA', 'NPS1_mean']],
    'SE1 Mean': [comparison.loc['High CA', 'SE1_mean'], comparison.loc['Low CA', 'SE1_mean']],
    'RLS1 Mean': [comparison.loc['High CA', 'RLS1_mean'], comparison.loc['Low CA', 'RLS1_mean']]
})
print(f"Median CA1 score: {ca_median}")
comparison_clean


In [ ]:
# Figure 7: High vs Low CA Comparison Bar Chart
fig, ax = plt.subplots(figsize=(10, 6))

outcome_labels = ['Growth (GO1)', 'Satisfaction (NPS1)', 'Self-Efficacy (SE1)', 'Relationship (RLS1/2)']
high_ca_means = [
    df[df['CA1_binary'] == 'High CA']['GO1'].mean(),
    df[df['CA1_binary'] == 'High CA']['NPS1'].mean(),
    df[df['CA1_binary'] == 'High CA']['SE1'].mean(),
    df[df['CA1_binary'] == 'High CA']['RLS1'].mean() / 2  # Scale to 1-5
]
low_ca_means = [
    df[df['CA1_binary'] == 'Low CA']['GO1'].mean(),
    df[df['CA1_binary'] == 'Low CA']['NPS1'].mean(),
    df[df['CA1_binary'] == 'Low CA']['SE1'].mean(),
    df[df['CA1_binary'] == 'Low CA']['RLS1'].mean() / 2  # Scale to 1-5
]

x = np.arange(len(outcome_labels))
width = 0.35

bars1 = ax.bar(x - width/2, high_ca_means, width, label=f'High CA (≥{ca_median:.0f})', color='#2E86AB')
bars2 = ax.bar(x + width/2, low_ca_means, width, label=f'Low CA (<{ca_median:.0f})', color='#E94F37')

ax.set_ylabel('Mean Score')
ax.set_title('Outcome Comparison: High vs Low Conflict Adaptation Teams')
ax.set_xticks(x)
ax.set_xticklabels(outcome_labels)
ax.legend()
ax.set_ylim(0, 5.5)

# Add value labels
for bar in bars1:
    ax.annotate(f'{bar.get_height():.2f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom', fontsize=9)
for bar in bars2:
    ax.annotate(f'{bar.get_height():.2f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()


## 5. Conflict Adaptation by Section


In [ ]:
# Table: CA1 and outcomes by section
section_stats = df.groupby('Section').agg({
    'CA1': ['mean', 'std', 'count'],
    'GO1': 'mean',
    'NPS1': 'mean',
    'RLS1': 'mean'
}).round(2)
section_stats.columns = ['CA1 Mean', 'CA1 Std', 'N', 'GO1 Mean', 'NPS1 Mean', 'RLS1 Mean']
section_stats


In [ ]:
# Figure 8: CA1 Distribution by Section
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Box plot
df.boxplot(column='CA1', by='Section', ax=axes[0])
axes[0].set_xlabel('Section')
axes[0].set_ylabel('Conflict Adaptation (CA1)')
axes[0].set_title('CA1 Score Distribution by Section')
plt.suptitle('')

# Bar chart of means with error bars
section_means = df.groupby('Section')['CA1'].mean()
section_stds = df.groupby('Section')['CA1'].std()
sections = section_means.index

axes[1].bar(sections, section_means, yerr=section_stds, capsize=5, color='#2E86AB', alpha=0.8)
axes[1].set_xlabel('Section')
axes[1].set_ylabel('Mean CA1 Score')
axes[1].set_title('Mean CA1 by Section (±1 SD)')
axes[1].axhline(y=df['CA1'].mean(), color='red', linestyle='--', label=f'Overall Mean: {df["CA1"].mean():.2f}')
axes[1].legend()

plt.tight_layout()
plt.show()


## Summary of Key Findings

**Key observations from Conflict Adaptation (CA1) analysis:**

1. **Distribution**: Most teams report moderate-to-high conflict adaptation scores (median around 8-9/10)

2. **Correlations**: CA1 shows positive correlations with all outcome variables - teams that handle conflict well tend to report better outcomes

3. **Strongest relationships**: 
   - CA1 correlates most strongly with RLS1 (relationship strength) and NPS1 (team satisfaction)
   - Constructive conflict handling appears linked to stronger interpersonal bonds

4. **High vs Low CA**: Teams with above-median CA1 scores show consistently higher means across all outcome metrics

5. **Section differences**: Some variation exists between sections in conflict adaptation patterns
